In [1]:
import numpy as np
import pandas as pd
import json
import pprint
import json
pp = pprint.PrettyPrinter(indent=4)

## ML-100k - 0.7 train / 0.3 test - sparse item

## ML-1M - 0.7 train / 0.3 test - sparse item

## ML-1m - 20e - 0.2 train/0.8 test

In [2]:
results = [
'../metrics/mfnn/2016-05-13_13.07.07_mfnn_ml-1m_e20_tt-0.2_task-19_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.07.24_mfnn_ml-1m_e20_tt-0.2_task-18_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.14_mfnn_ml-1m_e20_tt-0.2_task-3_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.42_mfnn_ml-1m_e20_tt-0.2_task-11_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.08.59_mfnn_ml-1m_e20_tt-0.2_task-1_user-metrics.csv',
'../metrics/mfnn/2016-05-13_13.13.32_mfnn_ml-1m_e20_tt-0.2_task-0_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.02.05_mfnn_ml-1m_e20_tt-0.2_task-7_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.19.45_mfnn_ml-1m_e20_tt-0.2_task-13_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.22.53_mfnn_ml-1m_e20_tt-0.2_task-12_user-metrics.csv',
'../metrics/mfnn/2016-05-13_14.54.52_mfnn_ml-1m_e20_tt-0.2_task-17_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.30.09_mfnn_ml-1m_e20_tt-0.2_task-20_user-metrics.csv',
'../metrics/mfnn/2016-05-13_16.36.33_mfnn_ml-1m_e20_tt-0.2_task-14_user-metrics.csv',
'../metrics/mfnn/2016-05-13_17.12.47_mfnn_ml-1m_e20_tt-0.2_task-21_user-metrics.csv',
'../metrics/mfnn/2016-05-13_17.53.02_mfnn_ml-1m_e20_tt-0.2_task-8_user-metrics.csv',
'../metrics/mfnn/2016-05-13_18.35.07_mfnn_ml-1m_e20_tt-0.2_task-23_user-metrics.csv',
'../metrics/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_user-metrics.csv',
'../metrics/mfnn/2016-05-13_20.08.12_mfnn_ml-1m_e20_tt-0.2_task-22_user-metrics.csv',
'../metrics/mfnn/2016-05-13_20.10.52_mfnn_ml-1m_e20_tt-0.2_task-15_user-metrics.csv',
'../metrics/mfnn/2016-05-13_22.18.50_mfnn_ml-1m_e20_tt-0.2_task-2_user-metrics.csv',
]

In [3]:
result_map = map(lambda p: (p, pd.read_csv(p)['auc'].mean()), results)
sorted(result_map, key=lambda t:t[1], reverse=True)

[('../metrics/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_user-metrics.csv',
  0.8978495667178319),
 ('../metrics/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_user-metrics.csv',
  0.8911205847799619),
 ('../metrics/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_user-metrics.csv',
  0.8889464453164606),
 ('../metrics/mfnn/2016-05-13_13.08.42_mfnn_ml-1m_e20_tt-0.2_task-11_user-metrics.csv',
  0.8830935640725851),
 ('../metrics/mfnn/2016-05-13_13.13.32_mfnn_ml-1m_e20_tt-0.2_task-0_user-metrics.csv',
  0.8804266743776397),
 ('../metrics/mfnn/2016-05-13_22.18.50_mfnn_ml-1m_e20_tt-0.2_task-2_user-metrics.csv',
  0.8781281670501629),
 ('../metrics/mfnn/2016-05-13_20.10.52_mfnn_ml-1m_e20_tt-0.2_task-15_user-metrics.csv',
  0.8779493811055779),
 ('../metrics/mfnn/2016-05-13_14.19.45_mfnn_ml-1m_e20_tt-0.2_task-13_user-metrics.csv',
  0.8776865504778436),
 ('../metrics/mfnn/2016-05-13_17.53.02_mfnn_ml-1m_e20_tt-0.2_task-8_user-metrics.csv',
  0.8774628776136015),
 ('../

In [4]:
with open('../models/mfnn/2016-05-13_16.17.19_mfnn_ml-1m_e20_tt-0.2_task-6_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-6',
    u'hit_threshold': 4,
    u'lr': 0.01,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 96,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': False,
    u'user

In [5]:
with open('../models/mfnn/2016-05-13_19.30.42_mfnn_ml-1m_e20_tt-0.2_task-10_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-10',
    u'hit_threshold': 4,
    u'lr': 0.03,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 128,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.01,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': True,
    u'use

In [6]:
with open('../models/mfnn/2016-05-13_13.08.49_mfnn_ml-1m_e20_tt-0.2_task-5_config.json') as f:
    pp.pprint(json.loads(f.read()))

{   u'ada_eps': 1e-06,
    u'adagrad': True,
    u'binarize': True,
    u'binarize_neg': 0,
    u'binarize_pos': 1,
    u'binarize_threshold': 1,
    u'd2v_model': u'doc2vec-models/2016-04-14_17.36.08_20e_pv-dbow_size50_lr0.025_window8_neg5',
    u'eval_in_parallel': False,
    u'experiment_name': u'mfnn_ml-1m_e20_tt-0.2_task-5',
    u'hit_threshold': 4,
    u'lr': 0.01,
    u'lr_decay': 0.02,
    u'metrics_save_dir': u'metrics/mfnn',
    u'model_save_dir': u'models/mfnn',
    u'nb_d2v_features': 50,
    u'nb_epochs': 20,
    u'nb_latent_f': 64,
    u'precision_recall_at_n': 20,
    u'ratings_path': u'data/splits/ml-1m/ratings.csv',
    u'reg_lambda': 0.003,
    u'run_eval': True,
    u'run_movie_metrics': False,
    u'sparse_item': True,
    u'test': True,
    u'test_path': u'data/splits/ml-1m/sparse-item/0.2-test.csv',
    u'top_n_predictions': 100,
    u'train_path': u'data/splits/ml-1m/sparse-item/0.2-train.csv',
    u'train_test_split': 0.2,
    u'use_avg_rating': False,
    u'use